<a target="_blank" href="https://colab.research.google.com/github/IngCarlaPezzone/tensorflow-1-public/blob/main/C1/W2/ungraded_labs/C1_W2_Lab_2_callbacks_traducida.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Ungraded Lab: Uso de las devoluciones de llamada (Callbacks) para controlar el entrenamiento

En este laboratorio, utilizará la [API de devoluciones de llamada](https://keras.io/api/callbacks/) para detener el entrenamiento cuando se cumpla una métrica especificada. Esta es una función útil para no tener que completar todas las épocas cuando se alcanza este umbral. Por ejemplo, si estableces 1000 épocas y la precisión deseada se alcanza en la época 200, el entrenamiento se detendrá automáticamente. Veamos cómo se implementa esto en las siguientes secciones.

## Cargar y normalizar el conjunto de datos Fashion MNIST

Al igual que en el laboratorio anterior, para este ejercicio volverás a utilizar el conjunto de datos Fashion MNIST. Y también como se mencionó antes, normalizará los valores de los píxeles para ayudar a optimizar el entrenamiento.

In [1]:
import tensorflow as tf

# Instantiate the dataset API
fmnist = tf.keras.datasets.fashion_mnist

# Load the dataset
(x_train, y_train),(x_test, y_test) = fmnist.load_data()

# Normalize the pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0

## Creación de una clase Callback

Puedes crear una devolución de llamada definiendo una clase que herede la clase base [tf.keras.callbacks.Callback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/Callback). A partir de ahí, puedes definir los métodos disponibles para establecer dónde se ejecutará el callback. Por ejemplo, a continuación, utilizarás el método [on_epoch_end()](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/Callback#on_epoch_end) para comprobar la pérdida en cada época de entrenamiento.

In [2]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        '''
        Detiene el entrenamiento tras alcanzar el 60% de precisión

        Args:
          epoch (integer) - índice de epoch (requerido pero no utilizado en la definición de la función)
          logs (dict) - resultados métricos de la época de entrenamiento
        '''

        # Check accuracy
        if(logs.get('loss') < 0.4):
            # Parada si se alcanza el umbral
            print("\nLoss is lower than 0.4 so cancelling training!")
            self.model.stop_training = True

# Instantiate class
callbacks = myCallback()

## Definir y compilar el modelo

A continuación, definirás y compilarás el modelo. La arquitectura será similar a la que construyó en el laboratorio anterior. Después, definirás el optimizador, la pérdida y las métricas que utilizarás para el entrenamiento.

In [3]:
# Define the model
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

# Compile the model
model.compile(optimizer=tf.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



### Entrenar el modelo

Ahora estás listo para entrenar el modelo. Para establecer el callback, simplemente establece el parámetro `callbacks` a la instancia `myCallback` que declaraste antes. Ejecuta la celda de abajo y observa lo que ocurre.

In [4]:
# Train the model with a callback
model.fit(x_train, y_train, epochs=10, callbacks=[callbacks])

Epoch 1/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4729 - accuracy: 0.8311
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3571 - accuracy: 0.8694

Loss is lower than 0.4 so cancelling training!


Notarás que el entrenamiento no necesita completar las 10 épocas. Al tener una llamada de retorno al final de cada época, es capaz de comprobar los parámetros de entrenamiento y comparar si cumple con el umbral establecido en la definición de la función. En este caso, simplemente se detendrá cuando la pérdida caiga por debajo de `0.40` después de la época actual.

*Reto opcional: Modificar el código para que el entrenamiento se detenga cuando la métrica de precisión supere el 60%.*

Con esto concluye este sencillo ejercicio sobre las devoluciones de llamada.